In [14]:
import backtrader as bt


def show_metrics(thestrats, cerebro):
    profit = cerebro.broker.getvalue() - cerebro.broker.startingcash
    per_profit = (profit / cerebro.broker.startingcash) * 100

    drawdown = thestrats[0].analyzers.drawdown.get_analysis()
    analysis = thestrats[0].analyzers.traders.get_analysis()
    print('Trades:', analysis['total']['total'])
    print('Trades Vencedores:', analysis['won']['total'])
    print('Ganho máximo:', analysis['won']['pnl']['max'])
    print('Ganho Médio:', analysis['won']['pnl']['average'])
    print('Perda Máxima:', analysis['lost']['pnl']['max'])
    print('Perda Média:', analysis['lost']['pnl']['average'])
    print('Drawdown Máximo:', drawdown.max.drawdown)
    print('Fator Lucro:', analysis['won']['pnl']['total'] / abs(analysis['lost']['pnl']['total']))
    print('Lucro: %.2f' % per_profit)
    print('Número de Stops Loss Atingidos: %d' % thestrats[0].stops_hit)


In [19]:
class BollingerBands(bt.Strategy):
    params = (
        ("period", 20),
        ("devfactor", 2),
        ("stop_loss", 0.012),
    )

    def __init__(self):
        self.bollinger = bt.indicators.BollingerBands(
            self.data, period=self.params.period, 
            devfactor=self.params.devfactor
        )
        self.buy_price = None
        self.stops_hit = 0

    def next(self):
        if not self.position:
            if (self.data.close[0] > self.bollinger.lines.bot[0]) and (
                self.data.close[-1] < self.bollinger.lines.bot[-1]
               ):
                self.buy()
        else:
            if self.data.close[0] > self.bollinger.lines.top[0]:
                self.close()
            elif self.data.close[0] <= self.buy_price * (1 - self.p.stop_loss):
                self.stops_hit += 1
                self.close()
                
    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

data = bt.feeds.GenericCSVData(
    dataname='data/WINM24_05_MIN.csv', 
    timeframe=bt.TimeFrame.Minutes, 
    compression=5
)

cerebro = bt.Cerebro()
cerebro.addstrategy(BollingerBands)
cerebro.adddata(data)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='traders')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.broker.set_cash(10000)
cerebro.broker.setcommission(commission=1.5, margin=120, mult=0.2)
thestrats = cerebro.run()
analysis = thestrats[0].analyzers.traders.get_analysis()
show_metrics(thestrats, cerebro)

Trades: 92
Trades Vencedores: 59
Ganho máximo: 309.0
Ganho Médio: 76.96610169491525
Perda Máxima: -470.0
Perda Média: -141.42424242424244
Drawdown Máximo: 15.952114458124482
Fator Lucro: 0.9730019284336833
Lucro: -1.26
Número de Stops Loss Atingidos: 7


In [18]:
class BollingerBandsStopMovel(bt.Strategy):
    params = (
        ("period", 20),
        ("devfactor", 2),
        ("initial_stop_percent", 0.008),
        ("stop_increment", 30)
    )

    def __init__(self):
        self.bollinger = bt.indicators.BollingerBands(
            self.data, period=self.params.period, 
            devfactor=self.params.devfactor
        )
        self.buy_price = None
        self.stop_price = None
        self.stops_hit = 0


    def next(self):
        if not self.position:
            if (self.data.close[0] > self.bollinger.lines.bot[0]) and (
                self.data.close[-1] < self.bollinger.lines.bot[-1]
               ):
                self.buy()
                self.stop_price = self.bollinger.lines.bot[0] * (1 - self.params.initial_stop_percent)
        else:
            if self.data.close[0] > self.data.open[0]:
                self.stop_price += self.params.stop_increment
                
            if self.position and self.data.close[0] > self.bollinger.lines.top[0]:
                self.close()
                
            elif self.data.close[0] <=  self.stop_price:
                self.stops_hit += 1
                self.close()

    def notify_order(self, order):
        if order.status in [order.Completed]:
            if order.isbuy():
                self.buy_price = order.executed.price

cerebro = bt.Cerebro()
cerebro.addstrategy(BollingerBandsStopMovel)

data = bt.feeds.GenericCSVData(
    dataname='data/WINM24_05_MIN.csv', 
    timeframe=bt.TimeFrame.Minutes, 
    compression=5
)
cerebro.adddata(data)
cerebro.addobserver(bt.observers.BuySell)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='traders')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.broker.set_cash(10000)
cerebro.broker.setcommission(commission=1.5, margin=120, mult=0.2)
thestrats = cerebro.run()
analysis = thestrats[0].analyzers.traders.get_analysis()
show_metrics(thestrats, cerebro)



Trades: 109
Trades Vencedores: 69
Ganho máximo: 309.0
Ganho Médio: 72.27536231884058
Perda Máxima: -312.0
Perda Média: -93.85
Drawdown Máximo: 9.229327711407825
Fator Lucro: 1.328449653702717
Lucro: 12.33
Número de Stops Loss Atingidos: 25


In [15]:
import backtrader.analyzers as btanalyzers
import pandas as pd
import backtrader as bt


cerebro = bt.Cerebro()

data = bt.feeds.GenericCSVData(
        dataname='data/WINM24_05_MIN.csv', 
        timeframe=bt.TimeFrame.Minutes, 
        compression=5
)

cerebro.adddata(data)

strats = cerebro.optstrategy(
        BollingerBandsStopMovel,
        initial_stop_percent = range(8, 9), 
        stop_increment = range(50, 51, 1)
)
cerebro.broker.setcash(10000)
cerebro.broker.setcommission(commission=1.5, margin=120, mult=0.2)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='traders')
cerebro.addanalyzer(btanalyzers.Returns, _name = "returns")
back = cerebro.run()

par_list = [[x[0].params.initial_stop_percent, 
             x[0].params.stop_increment,
             x[0].analyzers.traders.get_analysis['total']['total'],
             x[0].analyzers.traders.get_analysis ['won']['total'],
             x[0].analyzers.traders.get_analysis['won']['pnl']['average'],
             x[0].analyzers.traders.get_analysis['lost']['pnl']['average'],
             x[0].analyzers.returns.get_analysis()['rnorm100'], 
            ] for x in back]

par_df = pd.DataFrame(par_list, columns = [
    'initial_stop_percent', 'stop_increment', 'Traders', 
    'Traders Vencedores', 'Ganho Médio', 'perda Médio', 'Retorno'
])
par_df